In [1]:
import mne
import os
from os.path import abspath
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
# from mne.time_frequency import psd_welch
from tqdm import tqdm
import itertools
import warnings
# import pywt
from matplotlib import cm
from multiprocessing import Manager
warnings.filterwarnings("ignore")
from multiprocessing import Process

In [2]:
# path = "/media/kashraf/TOSHIBA EXT/Driving_dataset/23978112/20973_2_raw.fif"
# data= mne.io.read_raw_fif(path)

In [24]:
# mne.events_from_annotations(data)
# data.info.values()

In [3]:
file_path="/media/kashraf/Elements/data_gen_may_2021/EEG data/Audio/"
filename=os.listdir(file_path)
path_montage="/media/kashraf/Elements/data_gen_may_2021/montage/"
montage=mne.channels.read_montage(path_montage+"//"+"neuroscan64ch.loc")
raw_data=[]
for file in tqdm(filename):
    files=mne.io.read_raw_cnt(file_path+"/"+file,montage=montage, preload=True,verbose=False);
    raw_data.append(files)

## Selecting channels to include
good_ch= mne.pick_channels(raw_data[0].info['ch_names'], include=[],
                        exclude=["EKG","EMG",'VEO','HEO','Trigger'])
mne.pick_info(raw_data[0].info,sel=good_ch,copy=False,verbose=False)

for f in tqdm(raw_data):
    mne.pick_info(f.info,sel=good_ch,copy=False)
    f.set_montage(montage)

100%|██████████████████████████████████████████████████████████████████████████████████| 11/11 [00:00<00:00, 558.60it/s]


In [4]:
events= mne.events_from_annotations(raw_data[6])
events


Used Annotations descriptions: ['22', '24', '255', '26', '28', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45', '51', '52', '62', '64', '66', '68']


(array([[   639,      0,      3],
        [   821,      0,      3],
        [  1038,      0,      3],
        ...,
        [639996,      0,      3],
        [640492,      0,      3],
        [641200,      0,      3]]),
 {'22': 1,
  '24': 2,
  '255': 3,
  '26': 4,
  '28': 5,
  '31': 6,
  '32': 7,
  '33': 8,
  '34': 9,
  '35': 10,
  '36': 11,
  '37': 12,
  '38': 13,
  '39': 14,
  '40': 15,
  '41': 16,
  '42': 17,
  '43': 18,
  '44': 19,
  '45': 20,
  '51': 21,
  '52': 22,
  '62': 23,
  '64': 24,
  '66': 25,
  '68': 26})

In [5]:
import multiprocessing
from tqdm import tqdm
data=dict()
for i in range(len(raw_data)):
    data[i+1]={
    2:mne.Epochs(raw_data[i], events[0], event_id=1).get_data(),
    4:mne.Epochs(raw_data[i], events[0], event_id=2).get_data(),
    6:mne.Epochs(raw_data[i], events[0], event_id=4).get_data(),
    8:mne.Epochs(raw_data[i], events[0], event_id=5).get_data(),
    }

45 matching events found
Applying baseline correction (mode: mean)
Not setting metadata
0 projection items activated
Loading data for 45 events and 176 original time points ...
0 bad epochs dropped
45 matching events found
Applying baseline correction (mode: mean)
Not setting metadata
0 projection items activated
Loading data for 45 events and 176 original time points ...
0 bad epochs dropped
45 matching events found
Applying baseline correction (mode: mean)
Not setting metadata
0 projection items activated
Loading data for 45 events and 176 original time points ...
0 bad epochs dropped
45 matching events found
Applying baseline correction (mode: mean)
Not setting metadata
0 projection items activated
Loading data for 45 events and 176 original time points ...
0 bad epochs dropped
45 matching events found
Applying baseline correction (mode: mean)
Not setting metadata
0 projection items activated
Loading data for 45 events and 176 original time points ...
0 bad epochs dropped
45 matchin

0 projection items activated
Loading data for 45 events and 176 original time points ...
0 bad epochs dropped
45 matching events found
Applying baseline correction (mode: mean)
Not setting metadata
0 projection items activated
Loading data for 45 events and 176 original time points ...
0 bad epochs dropped
45 matching events found
Applying baseline correction (mode: mean)
Not setting metadata
0 projection items activated
Loading data for 45 events and 176 original time points ...
0 bad epochs dropped


In [9]:

# def compute_erp(sub_data, n_trials = 5):
#     batch_size = sub_data.shape[0]//n_trials
#     erps = np.zeros((batch_size,64,176))
#     for i in range(sub_data.shape[0]//n_trials):
#         start= n_trials*i
#         end = start+n_trials
# #         print(start,end)
#         batch_erp = sub_data[start:end].mean(axis=0)
#         erps[i] = batch_erp
        
#     return erps
# path = "/media/kashraf/Elements/Journal-2023/Data/audio/batch_erp/"
# for sub in tqdm(data):
#     batch_data = compute_erp(data[sub][6])
#     for i, batch_data in enumerate(batch_data):
#         np.save(os.path.join(path,"cl6",f"Sub_{sub}_Batch_{i}.npy"),batch_data)

100%|███████████████████████████████████████████| 11/11 [00:00<00:00, 26.52it/s]


In [116]:

def compute_erp(sub_data, n_trials = 5):
    batch_size = sub_data.shape[0]//n_trials
    if batch_size == 12:
        erps = np.zeros((batch_size,64,176))
        for i in range(sub_data.shape[0]//n_trials):
            start= n_trials*i
            end = start+n_trials
    #         print(start,end)
            batch_erp = sub_data[start:end].mean(axis=0)
            erps[i] = batch_erp
    else:
        erps = np.zeros((batch_size+1,64,176))
        for i in range(batch_size):
            start= n_trials*i
            end = start+n_trials
    #         print(start,end)
            batch_erp = sub_data[start:end].mean(axis=0)
            erps[i] = batch_erp
        
        erps[batch_size]= sub_data[55:sub_data.shape[0]].mean(axis=0)
        
    return erps
path = "/media/kashraf/Elements/Journal-2023/Data/visual/batch_erp"
for sub in tqdm(data):
    batch_data = compute_erp(data[sub][8])
    for i, batch_data in enumerate(batch_data):
        np.save(os.path.join(path,"cl8",f"Sub_{sub}_Batch_{i}.npy"),batch_data)

100%|███████████████████████████████████████████| 15/15 [00:00<00:00, 87.44it/s]


# Topomap generattion

In [10]:
info = raw_data[0].info
path ="/media/kashraf/Elements/Journal-2023/Data/audio/batch_erp/cl2"
path_topo = "/media/kashraf/Elements/Journal-2023/Data/audio/batch_topo/cl2"
# np_names = os.listdir(path)
# for file in tqdm(np_names):
#     data= np.load(path+file).reshape(1,64,176)
#     epoch = mne.EpochsArray(data,info= info)
#     psd,freq= mne.time_frequency.psd_welch(epoch,n_fft=176,verbose=0)
#     psd= np.average(psd,axis=2).flatten()
#     fig = mne.viz.plot_topomap(psd,pos=info,cmap="viridis",contours=False,show=False)
#     fig[0].get_figure().savefig(os.path.join(path_topo,file[:-3]+"png"),bbox_inches='tight', transparent=True,
#                                  pad_inches=0)

In [18]:
import os
import numpy as np
import mne
from multiprocessing import Pool
from tqdm import tqdm

def process_file(file):
    path = "/media/kashraf/Elements/Journal-2023/Data/audio/batch_erp/cl8"
    path_topo = "/media/kashraf/Elements/Journal-2023/Data/audio/batch_topo/cl8"

    data = np.load(os.path.join(path, file)).reshape(1, 64, 176)
    epoch = mne.EpochsArray(data, info=info)
    psd, freq = mne.time_frequency.psd_welch(epoch, n_fft=176, verbose=0)
    psd = np.average(psd, axis=2).flatten()
    fig= mne.viz.plot_topomap(psd, pos=info, cmap="viridis", contours=False, show=False)
    fig[0].get_figure().savefig(os.path.join(path_topo, file[:-3] + "png"), bbox_inches='tight', transparent=True, pad_inches=0)
    plt.close()  # Ensure you close the figure to free memory

def main():
    path = "/media/kashraf/Elements/Journal-2023/Data/audio/batch_erp/cl8"
    np_names = os.listdir(path)

    # Use as many processes as there are CPUs available
    with Pool(processes=os.cpu_count()) as pool:
        list(tqdm(pool.imap(process_file, np_names), total=len(np_names)))

if __name__ == "__main__":
    main()


  0%|                                                    | 0/99 [00:00<?, ?it/s]

1 matching events found
1 matching events found
1 matching events found
No baseline correction applied
1 matching events found
No baseline correction applied
1 matching events found
No baseline correction applied
1 matching events found
No baseline correction applied
No baseline correction applied
1 matching events found
Not setting metadata
No baseline correction applied
Not setting metadata
Not setting metadata
1 matching events found
1 matching events found
No baseline correction applied
No baseline correction applied
1 matching events found
0 projection items activated
No baseline correction applied
Not setting metadata
Not setting metadata
Not setting metadata
0 projection items activated
0 bad epochs dropped
No baseline correction applied
Not setting metadata
0 projection items activated
Not setting metadata
0 projection items activated
Not setting metadata
0 projection items activated
0 projection items activated
0 bad epochs dropped
Not setting metadata
0 projection items activ

  1%|▍                                           | 1/99 [00:00<00:50,  1.94it/s]

0 bad epochs dropped
Not setting metadata
1 matching events found
0 projection items activated
No baseline correction applied
1 matching events found
0 bad epochs dropped
No baseline correction applied
Not setting metadata
Not setting metadata
0 projection items activated
0 projection items activated
0 bad epochs dropped
0 bad epochs dropped


 11%|████▊                                      | 11/99 [00:00<00:03, 23.16it/s]

...

 80%|██████████████████████████████████▎        | 79/99 [00:03<00:01, 15.21it/s]

1 matching events found
No baseline correction applied
1 matching events found
Not setting metadata
1 matching events found
1 matching events found
0 projection items activated
No baseline correction applied
Not setting metadata
No baseline correction applied
No baseline correction applied
1 matching events found
0 projection items activated
0 bad epochs dropped
0 bad epochs dropped
1 matching events found
No baseline correction applied
1 matching events found
1 matching events found
No baseline correction applied
Not setting metadata
No baseline correction applied
Not setting metadata
No baseline correction applied
Not setting metadata
0 bad epochs dropped
0 projection items activated
0 projection items activated
Not setting metadata
Not setting metadata
0 projection items activated
Not setting metadata
0 projection items activated
Not setting metadata
0 projection items activated
0 projection items activated
0 bad epochs dropped
0 bad epochs dropped
0 projection items activated
0 bad

 83%|███████████████████████████████████▌       | 82/99 [00:03<00:01, 16.85it/s]

1 matching events found
0 bad epochs dropped
0 bad epochs dropped
No baseline correction applied
Not setting metadata
0 projection items activated
0 bad epochs dropped


100%|███████████████████████████████████████████| 99/99 [00:03<00:00, 26.02it/s]


In [87]:
"/media/kashraf/Elements/Journal-2023/Data/visual/batch_erp/cl2/Sub_1_Batch_0.npy"[:-3]

'/media/kashraf/Elements/Journal-2023/Data/visual/batch_erp/cl2/Sub_1_Batch_0.'